In [1]:
## Benchmark between C, C++, and Numpy
## Task: computing sum of squares

In [2]:
# C code ("mysum.c")

# long long mysum(int n, int* array) {
#     long long res = 0;
#     for (int i=0; i<n; i++)
#         res += array[i]*array[i];
#     return res;
# }

In [3]:
# C++ code ("mysum.cpp")
# just add [extern "C"] at the top of the code

# extern "C"

# long long mysum(int n, int* array) {
#     long long res = 0;
#     for (int i=0; i<n; i++)
#         res += array[i]*array[i];
#     return res;
# }

In [4]:
# save the following code snippet as setup.py at the working directory and run 'python setup.py build'
# to create the shared C/C++ modules

# from setuptools import setup, Extension

# setup(
#     ext_modules=[Extension('mysum_c', ['mysum.c'],),
#                  Extension('mysum_cpp', ['mysum.cpp'],),
#                  ],
#     install_requires=['numpy']
# )


In [5]:
# import libraries

import time
import numpy as np
import os

# required to import the C module
import ctypes

In [6]:
# compiled shared libraries
libfile_c = 'mysum_c.cpython-37m-darwin.so'
libfile_cpp = 'mysum_cpp.cpython-37m-darwin.so'

In [7]:
# import C and C++ modules

mylib_c = ctypes.CDLL(libfile_c)

mylib_c.mysum.restype = ctypes.c_longlong
mylib_c.mysum.argtypes = [ctypes.c_int,
                          np.ctypeslib.ndpointer(
                            dtype=np.int32
                          )]

mylib_cpp = ctypes.CDLL(libfile_cpp)

mylib_cpp.mysum.restype = ctypes.c_longlong
mylib_cpp.mysum.argtypes = [ctypes.c_int,
                            np.ctypeslib.ndpointer(
                              dtype=np.int32
                            )]

In [8]:
# data and number of simulations

array = np.arange(0, 1000000, 1, np.int32)
n_sim = 100

In [9]:
# compute execution times
start = time.time()
for i in range(n_sim):
    mylib_c.mysum(len(array), array)
end = time.time()
t_c = end-start

start_time_cpp = time.time()
for i in range(n_sim):
    mylib_cpp.mysum(len(array), array)
end_time_cpp = time.time()
t_cpp = end_time_cpp-start_time_cpp

start = time.time()
for i in range(n_sim):
    (array**2).sum()
end = time.time()
t_numpy = end-start

# This is similar
start = time.time()
for i in range(n_sim):
    pow(array, 2).sum()
end = time.time()
t_numpy_pow = end-start

# List comprehension
array = np.arange(0, 1000000, 1, np.int32)
start = time.time()
res = 0
for num in array:
    res += num**2
end = time.time()
t_listcomp = end-start


In [10]:
# print results
# C and C++ are about the same, and they are about

# They are similar
print(f'C: {t_c:.2f} seconds.')
print(f'C++: {t_cpp:.2f} seconds.')

# Slower than C/C++
print(f'Python_Numpy: {t_numpy:.2f} seconds.')

# about the same as using ** operator
print(f'Python_Numpy_with_pow(): {t_numpy_pow:.2f} seconds.')

# List comprehension is terribly slow
print(f'Python_Numpy_ListComp: {t_listcomp:.2f} seconds.')

C: 0.04 seconds.
C++: 0.04 seconds.
Python_Numpy: 0.07 seconds.
Python_Numpy_with_pow(): 0.07 seconds.
Python_Numpy_ListComp: 2.04 seconds.


In [11]:
# C and C++ are about 1.8 times faster than Numpy

print(round(t_numpy/t_c, 2))
print(round(t_numpy/t_cpp, 2))

1.74
1.88
